In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from catboost import Pool

from data import get_data
from train import train_loop

In [2]:
cats = [
    'repair',
    'night',
    'month',
    'weekday'
]

drop_cols = [
    'road_id',
    'road_km',
    'datetime'
]

In [3]:
train = get_data('data/raw/train1.csv')

In [5]:
train.drop(['data_id', 'year', 'road_km_int'], axis=1, inplace=True)

In [4]:
train.shape

(7343154, 14)

In [5]:
clf, score = train_loop(train.drop(drop_cols, axis=1), cats=cats, class_weights=[1, 2, 4])

Размер выборки для обучения:  (121219, 11)
Размер выборки для тестирования:  (30305, 11)
F1-score: 0.36391989639804095
     predicted  truth
0.0    30221.0  29337
1.0       84.0    807
2.0        NaN    161


In [7]:
test = get_data('data/raw/test1.csv', target=False)

In [8]:
test_pool = Pool(test.drop(drop_cols, axis=1), cat_features=cats)
test['target'] = clf.predict(test_pool).astype(int)

In [9]:
test['target'].value_counts()

0    7335035
1       8036
2         83
Name: target, dtype: int64

In [10]:
from time import time
prediction = test[['datetime','road_id','road_km','target']][test['target'] != 0]
prediction.to_csv(f'data/predictions/sol_{int(time())}.csv', index=False)